# Project 3 - Gender Classifier
### Authors: John Mazon, LeTicia Cancel, Bharani Nitalla

**Video:**

**Assignment:** Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can.

Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the devtest set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set.

How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?


In [31]:
# Libraries
import nltk
from nltk.corpus import names
import random
from nltk.classify import apply_features

## Load Data

Using the code found in chapter 6 of Natural Language Processing with Python we are going to do Supervised Classification using the Names corpus. We labeled each name by gender when loading the names files into a list and then shuffuled the list to make the list order random. We then check the length of the list and we can see that we have a total of 7,944 names and each name is labeled 'male' or 'female' based on which text file it was imported from. 

In [32]:
# Split male and female names
names = ([(name, 'male') for name in names.words('male.txt')] +
        [(name, 'female') for name in names.words('female.txt')])

In [33]:
# shuffle the names list in random order
random.shuffle(names)

In [34]:
names[:10]

[('Angelico', 'male'),
 ('Joice', 'female'),
 ('Karil', 'female'),
 ('Dodie', 'female'),
 ('Goober', 'male'),
 ('Cara', 'female'),
 ('Willa', 'female'),
 ('Vanessa', 'female'),
 ('Luke', 'male'),
 ('Marthena', 'female')]

In [35]:
len(names)

7944

## Classify Names Dataset

When creating our names classifier, we have to decide which name features are relevent in predicting if a name belongs to a male or female. We begin first by using exactly what is in the text book to see how this performs on our dataset before exploring features of our own. 

The gender_features function takes each name and takes three features we specified and loads them to a dictionary. The three features we will look at are last letter, length of name, and first name. We then test the funtion using the name John to make sure the features dictionary returns the correct information. 

In [36]:
def gender_features(word):
    features = {}
    features['last_letter'] = word[-1]
    features['word_len'] = len(word) 
    features['first_letter'] = word[0]
    return features

In [37]:
gender_features('John')

{'last_letter': 'n', 'word_len': 4, 'first_letter': 'J'}

Now that we see the function works for our test name John, we run the function for every name in the names list and save this dictionary to the featuresets variable. So every name in the list will have dictionary values similar to the above test we did with the name John. The data is split into a training and test set using an equal number of names for each set. We then use the training set to train a Naive Bayes classifier. When we print the accuracy of this classifier, we can see that it has a 77% accuracy. We will continue with this training set and then in part 2 we will add more features to the classifier so we can raise the accuracy score.  

In [38]:
featuresets = [(gender_features(n), g) for n,g in names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(classifier, test_set))

0.772


Using the show_most_imformative_features function we can see that the last letter is the most effective in distinguishing the gender. We can see the likelihood ratios for 5 letters. Names that end in the letter A are 38.6 times more likely to be a female name than male and names ending in the letter V are also more likely to be female. 

In [39]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     38.6 : 1.0
             last_letter = 'k'              male : female =     30.6 : 1.0
             last_letter = 'f'              male : female =     16.5 : 1.0
             last_letter = 'p'              male : female =     11.8 : 1.0
             last_letter = 'v'              male : female =     10.5 : 1.0


We modify the training and test sets using the apply_features function. According to the textbook this is the best way to store the results of the gender_features function when you have a large corpora. We will will apply the features in a different way later, but this was good practice. 

In [40]:
train_set = apply_features(gender_features, names[500:])
test_set = apply_features(gender_features, names[500:])

Now we split the names list according to the project guidlines, 500 test names, 500 devtest_names, and the remainder as training names. 

In [41]:
train_names = names[6900:]
devtest_names = names[500:1000]
test_names = names[:500]

We run the gender_features for each of the names sets, run the Naives Bayes classifier and then check the accuracy. The accuracy for the devtest_set is 77% which is slightly lower than the test_set classifier we inspected earlier. 

In [42]:
train_set = [(gender_features(n), g) for (n,g) in train_names]
devtest_set = [(gender_features(n), g) for (n,g) in devtest_names]
test_set = [(gender_features(n), g) for (n,g) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.752


## Error Analysis

Using the devtest_names set we do an error analysis to see how many many times the classifier incorrectly predicts name genders. All of the classifier errors are stored in the error variable and we can see that 115 incorrect predictions were made which is 22% of the devtest set. 

In [43]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))
print('Number of errors: ',len(errors))
print('Number of devtest names: ',len(devtest_names))
print('Error rate: ', len(errors)/len(devtest_names))

Number of errors:  124
Number of devtest names:  500
Error rate:  0.248


We examine each name in the error set so we can try to find patterns that can be used in the new gender_features function.

In [44]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print ('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Abagael                       
correct=female   guess=male     name=Alison                        
correct=female   guess=male     name=Bab                           
correct=female   guess=male     name=Barb                          
correct=female   guess=male     name=Brook                         
correct=female   guess=male     name=Carilyn                       
correct=female   guess=male     name=Carlynn                       
correct=female   guess=male     name=Carolan                       
correct=female   guess=male     name=Debor                         
correct=female   guess=male     name=Dot                           
correct=female   guess=male     name=Easter                        
correct=female   guess=male     name=Elsbeth                       
correct=female   guess=male     name=Evaleen                       
correct=female   guess=male     name=Fanchon                       
correct=female   guess=male     name=Fayth      

We modify the gender_features function to also look at vowels in each name, the first two letters, the last two letter, and the length of the name. Now we will perform all of the steps to see if we get better results. 

In [45]:
def gender_features(word):
    features = {}
    if any(vow in word for vow in 'aeiou'):
        features['vowel'] = True
    else:
        features['vowel'] = False
    features['last_letter'] = word[-1]
    features['letter_two'] = word[-2]
    features['word_len'] = len(word) 
    features['first_letter'] = word[0]
    features['second_letter'] = word[1]
    return features

The accuracy increases slightly from 77% to 78%

In [46]:
train_set = [(gender_features(n), g) for (n,g) in train_names]
devtest_set = [(gender_features(n), g) for (n,g) in devtest_names]
test_set = [(gender_features(n), g) for (n,g) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.75


When we do our error analysis the number of errors drops from 112 to 88 and the percentage drops from 22% to 17.8%. Since we do see an improvement we can use this for the test set. 

In [47]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))
print('Number of errors: ',len(errors))
print('Number of devtest names: ',len(devtest_names))
print('Error rate: ', len(errors)/len(devtest_names))

Number of errors:  125
Number of devtest names:  500
Error rate:  0.25


## Final Performance Test

Using the test_set we will check the accuracy and do an error analysis.

In [48]:
print(nltk.classify.accuracy(classifier, test_set))

0.776


In [49]:
errors = []
for (name, tag) in test_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))
print('Number of errors: ',len(errors))
print('Number of devtest names: ',len(test_names))
print('Error rate: ', len(errors)/len(test_names))

Number of errors:  112
Number of devtest names:  500
Error rate:  0.224


In [50]:
for (tag, guess, name) in sorted(errors): # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
    print ('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Aleen                         
correct=female   guess=male     name=Allyson                       
correct=female   guess=male     name=Annabel                       
correct=female   guess=male     name=Arden                         
correct=female   guess=male     name=Ariel                         
correct=female   guess=male     name=Brittan                       
correct=female   guess=male     name=Carmon                        
correct=female   guess=male     name=Cat                           
correct=female   guess=male     name=Chandal                       
correct=female   guess=male     name=Chloris                       
correct=female   guess=male     name=Chrystel                      
correct=female   guess=male     name=Crystal                       
correct=female   guess=male     name=Demeter                       
correct=female   guess=male     name=Dix                           
correct=female   guess=male     name=Dominique  

## Conclusion

While the testset had a higher accuracy score than the devtest, the number of error and the error percentage was also higher for the testset. This did come as a bit fof a surprise to me. Since the accuracy score is lower for the test set, I expected the error rate to be at or lower that the error rate of the devtest. It is possible that we need to find more better patterns in the error list so we know how to modify the feature function. 

I would start by modifying the feature method to look at the number of vowels present in the name instead of just a True/False if a vowel exists. 